In [1]:
import yfinance as yf

In [2]:
sp500 = yf.download("^GSPC", start="2022-01-01", end="2023-12-31")
vix = yf.download("^VIX", start="2022-01-01", end="2023-12-31")

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [3]:
print(sp500)

Price             Close         High          Low         Open      Volume
Ticker            ^GSPC        ^GSPC        ^GSPC        ^GSPC       ^GSPC
Date                                                                      
2022-01-03  4796.560059  4796.640137  4758.169922  4778.140137  3831020000
2022-01-04  4793.540039  4818.620117  4774.270020  4804.509766  4683170000
2022-01-05  4700.580078  4797.700195  4699.439941  4787.990234  4887960000
2022-01-06  4696.049805  4725.009766  4671.259766  4693.390137  4295280000
2022-01-07  4677.029785  4707.950195  4662.740234  4697.660156  4181510000
...                 ...          ...          ...          ...         ...
2023-12-22  4754.629883  4772.939941  4736.770020  4753.919922  3046770000
2023-12-26  4774.750000  4784.720215  4758.450195  4758.859863  2513910000
2023-12-27  4781.580078  4785.390137  4768.899902  4773.450195  2748450000
2023-12-28  4783.350098  4793.299805  4780.979980  4786.439941  2698860000
2023-12-29  4769.830078  

In [4]:
print(vix)

Price           Close       High        Low       Open Volume
Ticker           ^VIX       ^VIX       ^VIX       ^VIX   ^VIX
Date                                                         
2022-01-03  16.600000  18.540001  16.559999  17.600000      0
2022-01-04  16.910000  17.809999  16.340000  16.570000      0
2022-01-05  19.730000  20.170000  16.580000  17.070000      0
2022-01-06  19.610001  21.059999  19.080000  20.290001      0
2022-01-07  18.760000  20.799999  18.570000  19.850000      0
...               ...        ...        ...        ...    ...
2023-12-22  13.030000  13.960000  13.000000  13.720000      0
2023-12-26  12.990000  13.800000  12.960000  13.770000      0
2023-12-27  12.430000  13.040000  12.370000  13.020000      0
2023-12-28  12.470000  12.650000  12.380000  12.440000      0
2023-12-29  12.450000  13.190000  12.360000  12.550000      0

[501 rows x 5 columns]
